In [ ]:
目录: 1.位运算 2.排序 3.二分查找 4. 滑动窗口 5.回溯 6.动态规划

## 1. 位运算

基本上就是想到对32位(int数据)做加减乘除异或，范围是[-2^31, 2^31-1]
- 符号位为1后面都为0，起始值为-1了; 符号位为0后面都为0，起始值为0，所以正数表示范围要-1
- 左移b<<i位相当于乘以2^i，右移b>>i位相当于除以2^i
- 相同的数异或为0，由小写字母组成的单词可以用26位的bits表示，
- 偶数二进制位1的个数与一半相同

经典题型: 整数除法，二进制加法，二进制中1的个数，只出现一次的数字，最大单次长度乘积

In [ ]:
# 整数除法: 不能用乘法、除法、mod
def divide(dividend, divisor):
    # 初始条件: 越界则单独处理; 提前锁定结果的符号
    if dividend == -2**31 and vidisor == -1: return 2**31-1
    if dividend == 0: return 0
    sign = 1 if (dividend > 0 and divisor > 0) | (dividend < 0 and divisor < 0) else -1
    # 两种方法: 减法模拟除法; 直接从结果位数试
    # 1. 减法模拟除法: 令被除数每次扩大2倍，如果比除数大了说明当前这么多除数相加能组成结果的一部分
    x, y = abs(dividend), abs(divisor)
    res = 0
    while x >= y:
        temp, cnt = y, 1
        while x >= temp<<1:
            temp<<=1
            cnt<<=1
        x -= temp
        res += cnt
    return sign * res
print("divide:", divide(15, 6))

# 二进制加法
def addBinary(a, b):
    # 二进制解法: 两个二进制位异或类似于相加，如果&为0说明就为结果，如果&不为0说明再加上进位就为结果
    # 进位要左移一位
    a, b = int(a, 2), int(b, 2)
    t, c = a ^ b, (a & b) << 1
    while c:
        t, c = t ^ c, (t & c) << 1
    return bin(t)[2:]
    
    # 传统做法，对字符串从末尾开始向前不断相加
    carry = 0
    n1, n2 = len(a), len(b)
    i, j, res = n1 - 1, n2 - 1, ""
    while i >= 0 or j >= 0 or carry:
        a1 = int(a[i]) if i >= 0 else 0
        b1 = int(b[j]) if j >= 0 else 0
        temp =  a1 + b1 + carry
        if temp >= 2:
            carry = 1
            temp -= 2
        else:
            carry = 0
        res = str(temp) + res
        i -= 1
        j -= 1
    return res
print("addBinary", addBinary("1010", "1011"))

# 二进制中1的个数
def countBits(n):
    # 偶数的二进制1的个数与它的一半相同，奇数的二进制1的个数=(奇数-1)//2+1
    # 求1~n所有数的1的个数
    res = [0] * (n + 1)
    for i in range(1, n + 1):
        if i % 2:
            res[i] = res[(i - 1) // 2] + 1
        else:
            res[i] = res[i // 2]
    return res
    '''
    # 求某一个数中1的个数
    if n == 1: return 1
    if not n % 2:
        return countBits(n // 2)
    else:
        return countBits((n - 1) // 2) + 1
    '''
print("countBits:", countBits(5))

# 数组中只出现一次的数字
def singleNumber(nums):
    # 两种解法: 1.循环一遍统计频率(哈希表)，但要额外空间  
    # 2.位运算，数字电路？! https://leetcode-cn.com/problems/WGki4K/solution/zhi-chu-xian-yi-ci-de-shu-zi-by-leetcode-0vrt/
    pass

# 最大单词长度乘积
def maxProduct(words) -> int:
    # 把每个单词转化成26位的二进制表示(通过位运算)
    # 若两个二进制表示&为0说明不包含相同字符，判断长度的乘积
    res = 0
    words_bi = []
    for word in words:
        temp = 0
        for c in word:
            temp = temp | 1 << (ord(c) - 97)
        words_bi.append(temp)
    for i in range(len(words)):
        for j in range(i + 1, len(words)):
            if not (words_bi[i] & words_bi[j]):
                res = max(res, len(words[i]) * len(words[j]))
    return res
print("maxProduct: ", maxProduct(["abcw","baz","foo","bar","xtfn","abcdef"]))



## 2. 排序 

冒泡排序，选择排序，插入排序，快速排序，归并排序，堆排序，计数排序, 拓扑排序

找第K大元素，合并区间，链表排序，相对排序

https://www.cs.usfca.edu/~galles/visualization/ComparisonSort.html



In [ ]:
# 冒泡排序
def bubbleSort(arr):
    # 最简单也最慢，通过不断交换将最大值不断往后送。优化就是设置flag，如果每一次没有发生交换说明已经排好了
    for i in range(len(arr)):
        # 每经过一次i最大的i个就被交换到最后面
        for j in range(len(arr) - i - 1):
            # 如果后一项比前一项小，就把大的交换到后面去
            if arr[j+1] < arr[j]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr

# 选择排序
def selectSort(arr):
    # 每遍历一次记住一个最小值的下标，跟最前面的进行交换。虽然也是进行了n²次比较，但是减少了交换的次数
    for i in range(len(arr)):
        min_index = i
        for j in range(i + 1, len(arr)):
            if arr[j] < arr[min_index]:
                min_index = j
        arr[i], arr[min_index] = arr[min_index], arr[i]
    return arr

# 插入排序
def insertSort(arr):
    # 假设一开始只有第一个元素，是排好序的，依次将后面的元素插入这个排序好的数组中。每插入一个元素都交换到正确的位置
    for i in range(len(arr)):
        while i > 0 and arr[i] < arr[i-1]:
            arr[i], arr[i-1] = arr[i-1], arr[i]
            i -= 1
    return arr

# 归并排序
def mergeSort(arr, left, right):
    # 前闭后闭
    if left == right: return
    mid = (left + right) // 2
    mergeSort(arr, left, mid)
    mergeSort(arr, mid+1, right)
    temp = []
    # 两个排序好的子区间的左边界
    i, j = left, mid + 1
    while i <= mid or j <= right:
        # 如果右边界出界了说明剩下的都是增加i这一部分；或者两边都没出界，i这边的偏小
        if j > right or (i <= mid and arr[i] < arr[j]):
            temp.append(arr[i])
            i += 1
        # 剩下的情况是左边界出界了，或者右边这个元素较小
        else:
            temp.append(arr[j])
            j += 1
    arr[left: right+1] = temp


# 快速排序
def quick_sort(left, right):
    if left > right: return
    i, j = left, right
    # 设置基准点, 特别注意: 设置左端基准点要从右边开始查找，设置右基准点要从左边开始查找
    # 这是因为如果先动左边，本来是要找比基准点大的值，但是可能越界，导致停止点比基准点大，这样就被交换到基准点而永远不会去到右边
    pivot = arr[left]
    while i < j:
                 # 升序降序只要改 ↓ 这个符号就行了
        while i < j and arr[j] >= pivot:
            j -= 1
        while i < j and arr[i] <= pivot:
            i += 1
        arr[i], arr[j] = arr[j], arr[i]
    arr[i], arr[left] = arr[left], arr[i]
    quick_sort(left, i-1)
    quick_sort(i+1, right)

    
# 找第k大的元素
def findKthLargest(left, right, K):
    # 三种方法:
    # 1.快速排序: 每一次结果都是基准值左边都小于他，右边都大于他，因此根据基准值的位置可以判断目标值在左边还是右边
    # 2.整体排序: 全部排序再直接返回第K大的
    # 3.最小堆: 将数组中全部元素添加进最小堆，再返回第K个
    if left > right: return
    i, j = left, right
    pivot = arr[left]
    while i < j:
        while i < j and arr[j] <= pivot:
            j -= 1
        while i < j and arr[i] >= pivot:
            i += 1
        arr[i], arr[j] = arr[j], arr[i]
    arr[i], arr[left] = arr[left], arr[i]
    if i == K - 1:
        return arr[i]
    elif i > K - 1:
        return findKthLargest(left, i - 1, K)
    elif i < K - 1:
        return findKthLargest(i + 1, right, K)

'''
arr = [1, 4, 2, 6, 7, 8, 3]
quick_sort(0, len(arr) - 1)
print(arr)
print(findKthLargest(0, len(arr) - 1, 4))
print(bubbleSort(arr))
print(selectSort(arr))
print(insertSort(arr))
'''
arr = [1, 4, 2, 6, 7, 8, 3]
mergeSort(arr, 0, len(arr) - 1)
print(arr)

In [ ]:
# 堆排序(优先队列)
# 数据结构是完全二叉树，每一个节点都小于等于它子树中每个节点值(小顶堆)，可以用数组来储存
# 以0开始的数组中节点i的左孩子下标为2*i+1，右孩子2*i+2，节点的父节点(i-1)//2
# 主要步骤：建堆(上浮和下沉)，排序(将堆顶元素和最后一个元素交换，再执行下沉)
# https://leetcode-cn.com/problems/sort-an-array/solution/dong-hua-mo-ni-yi-ge-po-dui-pai-wo-gao-l-i6mt/
# https://www.jianshu.com/p/d174f1862601
def heapify(arr, n, i): 
    largest = i  
    l = 2 * i + 1     # left = 2*i + 1 
    r = 2 * i + 2     # right = 2*i + 2 
    if l < n and arr[i] < arr[l]: 
        largest = l 
    if r < n and arr[largest] < arr[r]: 
        largest = r 
    if largest != i: 
        arr[i],arr[largest] = arr[largest],arr[i]  # 交换
        heapify(arr, n, largest)

        
def heapSort(arr): 
    n = len(arr) 
    # Build a maxheap. 
    for i in range(n, -1, -1): 
        heapify(arr, n, i) 
    # 一个个交换元素
    for i in range(n-1, 0, -1): 
        arr[i], arr[0] = arr[0], arr[i]   # 交换
        heapify(arr, i, 0) 

arr = [1, 4, 2, 6, 7, 8, 3]
heapSort(arr)
arr

## 3. 二分查找
在有序序列中进行查找，主要是找target，找左边界，找右边界

可以设题目要求的变量x已知，求目标值随着x如何变化来调整x的范围 https://leetcode-cn.com/problems/minimized-maximum-of-products-distributed-to-any-store/

In [1]:
def binary_search(arr, target):
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] > target:
            right = mid - 1
        elif arr[mid] < target:
            left = mid + 1
    return -1

# bisect_left
# 如果有target(可能重复)，就返回最左边的那个；如果没有，就返回比他大的第一个元素(也就是这个数左边都小于target)
def binary_left_boundry(arr, target):
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        print(left, right, mid)
        if arr[mid] == target:
            right = mid - 1
        elif arr[mid] > target:
            right = mid - 1
        elif arr[mid] < target:
            left = mid + 1
    if left == len(arr):
        print("all elements are smaller than target")
    return left
    
# bisect_right
# 如果有target(可能重复)，就返回最右边的那个；如果没有，就返回比他小的第一个元素（也就是这个数右边都大于target）
def binary_right_boundry(arr, target):
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            left = mid + 1
        elif arr[mid] < target:
            left = mid + 1
        elif arr[mid] > target:
            right = mid - 1
    if right < 0:
        print("all elements are greater than target")
    return right
        

arr = [0, 1, 2, 3, 3, 4, 5, 7]
print(binary_search(arr, 6))
print(binary_left_boundry(arr, 3))
print(binary_right_boundry(arr, 3))

-1
0 7 3
0 2 1
2 2 2
3
4


## 4. 滑动窗口（双指针）

求链表中点

经常与数组连用，关键字: 求连续子数组，排序数组，数组内和/积小于target

In [ ]:
# 乘积小于K的子数组数目
def numSubarrayProductLessThanK(nums, K):
    # 两种方法: 
    # 1. 前缀积。看前面有多少个数满足 <= 当前积 / K，与二分查找连用。(会超时)
    # 2. 双指针。一步一步移动右指针，统计以当前右指针为右边界的数组有多少个满足条件的
    #          (左指针移动到滑动窗口乘积 < K) 此时数量为(right - left + 1)
    res = 0
    left, temp = 0, 1
    for right in range(len(nums)):
        temp *= nums[right]
        while left < right and temp >= K:
            temp /= nums[left]
            left += 1
        res += (right - left + 1)
    return res

## 5. 回溯

79-97
可用于数组、字符串、树、图

中心想法: 每个位置的元素取还是不取   常见变量: path, index, root, 

一般步骤: 确定每次递归改变的变量，确定终止条件(越界)，确定满足条件的操作，处理当前节点后如何进行下一步递归

经典问题: 找子集、找路径、匹配括号、分割回文串、(不)含重复元素的排列组合、(不)含重复元素的全排列、复原IP地址

关键步骤: 重复元素的话某个位置相同的元素只能出现一次，全排列的话所有元素都要出现一次

In [2]:
# 所有子集 (数组中元素互不相同，返回所有子集)
def subsets(nums):
    res = []
    def dfs(path, index):
        if index == len(nums): 
            res.append(list(path))
            return
        # 取或者不取
        path.append(nums[index])
        dfs(path, index+1)
        path.pop()
        dfs(path, index+1)
    dfs([], 0)
    return res
print("subsets: ", subsets([1,2,3]))

# k个组合 (返回[1,n]范围内所有k个数的组合)
def combine(n, k):
    res = []
    def dfs(path, index):
        if len(path) == k:
            res.append(list(path))
            return
        if index == n:
            return
        path.append(index)
        dfs(path, index+1)
        path.pop()
        dfs(path, index+1)
    dfs([], 1)
    return res
print("combine: ", combine(4, 2))

# 组合总和 (无重复元素的正整数数组中找到数字和为target的所有组合，各个数字可以被无限制选取) (无重复元素数组，允许重复选择元素)
def combinationSum(candidates, target):
    res = []
    def dfs(path, index, total):
        if index == len(candidates) or total > target:
            return
        if total == target:
            res.append(list(path))
            return
        path.append(candidates[index])
        dfs(path, index, total+candidates[index])
        path.pop()
        dfs(path, index+1, total)
    dfs([], 0, 0)
    return res
print("combinationSum: ", combinationSum([2,3,5], 8))

# 组和总和2 (有重复元素的正整数数组中找到数字和为target的所有组合) (含有重复元素的数组，数字只能使用一次)
# 不能包含重复组合([1,7]和[7,1]是重复的)
def combinationSum2(candidates, target):
    candidates.sort()
    res = []
    def dfs(path, index, total):
        # 要先判断是否能够加入，不然[1,2,5]的5加不进来
        if total == target:
            res.append(list(path))
            return
        if index == len(candidates) or total > target:
            return
        # 每一次递归代表一个位置的选择。
        # 对于当前位置，不能有重复元素出现，并且后面的元素无法使用前面的元素。所以必须要排序,否则7还可以用它后面的1
        visited = []
        for i in range(index, len(candidates)):
            if candidates[i] not in visited:
                visited.append(candidates[i])
                path.append(candidates[i])
                dfs(path, i+1, total+candidates[i])
                path.pop()
    dfs([], 0, 0)
    return res
print("combinationSum2: ", combinationSum2([10,1,2,7,6,1,5], 8))

# 全排列 (无重复元素的数组，返回所有可能的全排列) (没有重复元素集合的全排列)
def permute(nums):
    res = []
    visited = [0] * len(nums)
    def dfs(path):
        if len(path) == len(nums):
            res.append(list(path))
            return
        for i in range(len(nums)):
            if not visited[i]:
                visited[i] = 1
                path.append(nums[i])
                dfs(path)
                visited[i] = 0
                path.pop()
    dfs([])
    return res
print("permute: ", permute([1,2,3]))

# 全排列2 (返回不重复的全排列) (含有重复元素的全排列)
def permuteUnique(nums):
    # 同一个位置不能出现一样的元素, 并且加入的数字不用再访问
    nums.sort()
    res = []
    visited = [0] * len(nums)
    def dfs(path):
        if len(path) == len(nums):
            res.append(list(path))
            return
        temp = []
        for i in range(len(nums)):
            if nums[i] not in temp and not visited[i]:
                visited[i] = 1
                temp.append(nums[i])
                path.append(nums[i])
                dfs(path)
                path.pop()
                visited[i] = 0
    dfs([])
    return res
print("permuteUnique: ", permuteUnique([1,1,2]))

# 复原IP地址 (将字符串分为四分，不含有前导0，有有效的IP地址)
def restoreIPAddresses(s):
    res = []
    def dfs(path, index):
        if len(path) == 4 and index == len(s):
            res.append(".".join(path))
            return
        if len(path) > 4 or index >= len(s):
            return
        for j in range(index + 1, min(index + 4, (len(s) + 1))):
            temp = s[index: j]
            # 不含有前导0且在0~255范围内；注意“0” <= "35" <= “255”是False，因为字符串比较是从首位开始比较
            if len(temp) == len(str(int(temp))) and 0<= int(temp) <= 255:
                path.append(temp)
                dfs(path, j)
                path.pop()
    dfs([], 0)
    return res
print("restoreIPAddresses: ", restoreIPAddresses("25525511135"))


subsets:  [[1, 2, 3], [1, 2], [1, 3], [1], [2, 3], [2], [3], []]
combine:  [[1, 2], [1, 3], [2, 3]]
combinationSum:  [[2, 2, 2, 2], [2, 3, 3], [3, 5]]
combinationSum2:  [[1, 1, 6], [1, 2, 5], [1, 7], [2, 6]]
permute:  [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 1, 2], [3, 2, 1]]
permuteUnique:  [[1, 1, 2], [1, 2, 1], [2, 1, 1]]
restoreIPAddresses:  ['255.255.11.135', '255.255.111.35']


## 6. 动态规划

关键是找到状态以及状态转移方程，看当前的状态与子问题的解是否有关联，经常出现于数组、字符串问题

本质就是自底向上，带记忆化的回溯(自顶向下): 比如斐波那契数列。回溯的时间复杂度是2^n，如果数据量小的话可以

https://leetcode-cn.com/problems/target-sum/solution/xi-wang-yong-yi-chong-gui-lu-gao-ding-bei-bao-we-2/

https://leetcode-cn.com/problems/combination-sum-iv/solution/dai-ma-sui-xiang-lu-377-zu-he-zong-he-iv-pj9s/

- 当前状态完全取决于前项: 爬楼梯，斐波那契

- 当前状态需要分情况: 
    * 房屋偷盗(偷不偷当前房屋)
    * 粉刷房子(当前房子涂成什么颜色)
    * 翻转字符(当前翻不翻转)

- 当前状态由两个变量的子问题共同决定:
    * 最长公共子序列(判断当前元素是否相同，记录目前状态相同元素的个数)
    * 字符串交织(判断当前元素来自哪个子串，记录目前状态是否能能拼接成target)
    * s中子序列为t的数目(判断当前元素是否相同，相同的话是否加入，记录目前状态匹配的个数)

#### 如果求组合数就是外层for循环遍历物品，内层for遍历背包。如果求排列数就是外层for遍历背包，内层for循环遍历物品。
   
- 0-1背包问题:
    * 分割等和子集(找到一个目标和为sum/2的子序列)
    * 加减目标和(找到一定数量的子序列使得正数和为目标值)
    * True / False问题: 单词拆分
    * 最大最小问题: 

- 完全背包问题
    * 组合问题: 兑换零钱(顺序无关)
    * 排列问题: 排列的数目(顺序有关)


### 动态规划 Part1: 单变量问题

In [ ]:
# 爬楼梯最少成本(每次只能爬一步或两步)
def minCostClimbingStairs(cost):
    # 状态: 走到当前这一步需要多少花费
    # 状态转移: 当前花费 = min(走到i-1的花费 + i-1的花费, 走到i-2步的花费 + i-2的花费)
    first, second = 0, 0
    for i in range(2, len(cost) + 1):
        first, second = second, min(first + cost[i-2], second + cost[i-1])
    return second
print("minCostClimbingStairs: ", minCostClimbingStairs([10, 15, 20]))

# 房屋偷盗（不能偷相邻的房屋）（1 <= nums.length <=100）
def rob(nums):
    # 状态: 到当前房屋最多能偷多少钱
    # 状态转移: 偷当前房屋，则不能偷前一个房屋，dp[i-2] + i；不偷当前房屋，则可以偷前一个房屋，dp[i-1]
    #         当前能偷的钱 = max(dp[i-2] + i, dp[i-1])
    first, second = 0, 0
    for i in range(len(nums)):
        first, second = second, max(first + nums[i], second)
    return second
print("rob: ", rob([1,2,3,1]))

# 环形房屋被盗 (首尾相连)
def robCircle(nums):
    # 类似上一题，但是分类讨论
    #  偷第一间房子: 只能从第三间到倒数第二间可以偷 /不偷第一间房子: 从第二间到最后一间都可以偷
    if len(nums) == 1: return nums[0]
    if len(nums) == 2: return max(nums[0], nums[1])
    return max(rob(nums[2:-1] + nums[0]), rob(nums[1:]))
print("robCircle: ", robCircle([2,3,2]))

# 粉刷房子 (每个房子可以粉刷成三种颜色，每个房子每种颜色不同价格，相邻房子不能一样)
def minCostPaint(costs):
    # 状态: 当前房子粉刷成各种颜色时的最小花费
    # 状态转移: 如果当前选红，cost = min(上一个蓝，上一个绿) + 当前红的花费
    red, blue, green = 0, 0, 0
    for cost in costs:
        red, blue, green = cost[0]+min(blue, green), cost[1]+min(red, green), cost[2]+min(red, blue)
    return min(red, blue, green)
print("minCostPaint: ", minCostPaint([[17,2,17],[16,16,5],[14,3,19]]))

# 翻转字符 (00、01、11称作单调递增，求字符串最少反转多少次才单调递增)
def minFlipsMonoIncr(s):
    # 状态: 当前反转最小的翻转次数，当前不反转最小的翻转次数
    # 状态转移: 如果当前为0，不反转的话，那么前一项必须是 1且反转 或者是 0且不反转
    #                     反转的话，那么无论前一项是什么都是递增的，次数为min(flip，noflip) + 1
    #          如果当前为1
    #                    不反转的话，那么无论前一项是什么都是递增的，次数为min(flip，noflip)
    #                     反转的话，那么前一项必须是 1且反转 或者是 0且不反转，对应的次数 + 1
    flip, noflip = 1, 0
    for i in range(1, len(s)):
        if s[i] == "0":
            if s[i-1] == "1":
                flip, noflip = min(flip, noflip) + 1, flip
            else:
                flip, noflip = min(flip, noflip) + 1, noflip
        else:
            if s[i-1] == "1":
                flip, noflip = flip + 1, min(flip, noflip)
            else:
                flip, noflip = noflip + 1, min(flip, noflip)
    return min(flip, noflip)

# 数组中能形成的最长斐波那契数列 (求严格递增的数组最长的fibonacci子数列)
def lenLongestFibSubseq(arr):
    # 从后往前，确定后两个数字，可以倒推回前面的数字
    # 状态: dp[i][j]记录以i, j开头，后面有多少个数字能组成斐波那契数列
    # 状态转移: 如果arr[i] == arr[k] - arr[j]，那么dp[i][j] = dp[j][k] + 1
    # 优化: 因为各个数字唯一，使用哈希表记录出现数字的下标，快速判断j之前是否存在i满足条件
    pass


### 动态规划 Part2: 多变量问题

In [2]:
# 最长公共子序列
def longestCommonSubsequence(text1, text2):
    # 最经典的双变量！
    # 状态: dp[i][j]表示text1[:i]与text2[:j]公共子序列的长度
    # 状态转移: 如果text1[i] == text2[j]，那么dp[i][j] = dp[i-1][j-1] + 1
    #         如果不相等，dp[i][j] = max(dp[i-1][j], dp[i][j-1])
    # 初始化: n+1, m+1的矩阵. 想象两个数组为"a", "a"的话怎么弄
    m, n = len(text1), len(text2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if text1[i - 1] == text2[j - 1]:
                dp[i][j] = dp[i-1][j-1] + 1
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])
    return dp[-1][-1]
print("longestCommonSubsequence: ", longestCommonSubsequence("abcde", "ace"))

# 字符串交织 (判断s3是否能由s1和s2交错组成)
def isInnerleave(s1, s2, s3):
    # 状态: dp[i][j]表示s1[:i]与s2[:j]是否能组成s3[:(i+j)]
    # 状态转移: 如果s3[i+j] == s1[i]，那么dp[i][j] = dp[i-1][j] 
    #         如果s3[i+j] == s2[j]，那么dp[i][j] = dp[i][j-1]   
    # 初始化: dp[0][0] = True
    n1, n2, n3 = len(s1), len(s2), len(s3)
    if n1 + n2 != n3: return False
    dp = [[True] * (n2 + 1) for _ in range(n1 + 1)]
    for i in range(1, n1 + 1):
        dp[i][0] = dp[i-1][0] and s1[i-1] == s3[i-1]
    for j in range(1, n2 + 1):
        dp[0][j] = dp[0][j-1] and s2[j-1] == s3[j-1]
    for i in range(1, n1 + 1):
        for j in range(1, n2 + 1):
            dp[i][j] = (dp[i-1][j] and s3[i+j-1] == s1[i-1]) or (dp[i][j-1] and s3[i+j-1] == s2[j-1])
    return dp[-1][-1]
print("isInnerleave: ", isInnerleave("aabcc", "dbbca", "aadbbcbcac"))

# 子序列数目(求s的子序列中t出现的个数)
def numDistict(s, t):
    # 状态: dp[i][j]表示s[:i]的子序列中包含多少t[:j]
    # 状态转移: 如果s[i] == t[j]，那么可以选择取或者不取，取的话dp[i][j] += dp[i-1][j-1]
    #                                              不取的话dp[i][j] += dp[i-1][j]
    #         如果s[i] != t[j]，dp[i][j] += dp[i-1][j]
    # 初始化: dp[i][0] = 1，目标串为空时，不管s是什么都能出现1次
    # 优化: 注意到状态转移方程其实只与dp[i-1]前一行有关
    # 优化版本
    m, n = len(s), len(t)
    if m < n: return 0
    dp = [1] + [0] * n
    for i in range(m):
        for j in range(n, 0, -1):
            if s[i] == t[j-1]:
                dp[j] += dp[j-1]
    return dp[-1]

    # 二维矩阵版本
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(m + 1):
        dp[i][0] = 1
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            dp[i][j] += dp[i-1][j]
            if s[i-1] == t[j-1]:
                dp[i][j] += dp[i-1][j-1]
    return dp[-1][-1]
print("numDistict: ", numDistict("babgbag", "bag"))

# 路径的数目 (m * n的网格，只能向下或向右，从左上角走到右下角有几种不同路径)
def uniquePath(m, n):
    # 可以用数学做: 从(m+n-2)里选出(m-1)个位置向下走。也可以用动态规划做。
    # 状态: dp[i][j]表示走到(i * j)的网格有几条不同路径
    # 状态转移: 可以从上也可以从左走来，dp[i][j] = dp[i-1][j] + dp[i][j-1]
    # 初始化: dp[0][j]和dp[i][0]都为1 (只有一条路径可到)
    dp = [[1] * n for _ in range(m)]
    for i in range(1, m):
        for j in range(1, n):
            dp[i][j] = dp[i-1][j] + dp[i][j-1]
    return dp[-1][-1]
print("uniquePath: ", uniquePath(3, 2))
 
# 最小路径之和 (m * n的网格, 只能向下或向右，找到一条左上角到右下角的路径使得路径总和最小)
def minPathSum(grid):
    # 状态: dp[i][j]表示走到(i * j)的网格最小路径和
    # 状态转移: dp[i][j] = min(dp[i-1][j], dp[i][j-1])
    # 初始化: 第一行累加，第一列累加
    m, n = len(grid), len(grid[0])
    for i in range(1, m ):
        grid[i][0] = grid[i-1][0] + grid[i][0]
    for j in range(1, n):
        grid[0][j] = grid[0][j-1] + grid[0][j]
    for i in range(1, m):
        for j in range(1, n):
            grid[i][j] = min(grid[i-1][j], grid[i][j-1]) + grid[i][j]
    return grid[-1][-1]
print("minPathSum: ", minPathSum([[1,3,1],[1,5,1],[4,2,1]]))

# 三角形中最小路径之和 (三角形自顶向下的最小路径和，每一次只能移动到下一层index相同或index+1的位置)
def minimumTotal(triangle):
    pass

longestCommonSubsequence:  3
isInnerleave:  True
numDistict:  5
uniquePath:  3
minPathSum:  7


### 动态规划Part3: 0-1背包问题

In [ ]:
# 分割等和子集 (能否将非空正数数组nums分为元素和相等的两部分)
def canPartition(nums):
    # 转化成: 从nums中选择几个元素组合，和为sum(nums)//2
    # 状态: dp[i][j]表示可选元素为nums[:i]，目标和为j
    # 状态转移: 如果选择当前元素, dp[i][j] = dp[i-1][j-nums[i]]，
    #         如果不选择当前元素, dp[i][j] = dp[i-1][j]
    # 初始化: dp[0][0] = True
    # 优化: 看状态转移方程只与上一行有关，逆序更新(dp[j]＝正上方或左上角某一个数)(确保左上角的数还是左上角的数)
    total = sum(nums)
    if total % 2: return False
    target = total // 2
    dp = [True] + [False] * target
    for num in nums:
        for j in range(target, -1, -1):
            if j >= num:
                dp[j] = dp[j - num] or dp[j]
    return dp[-1]

# 加减的目标值 (每个整数前可以添加+/-，求和为目标值的不同表达式数目)
def findTargetSumWays(nums, target):
    # 因为所有正数项和为x，负数项和为y，则 x = sum(nums) - y; 又有 x - y = target，可得 x = (sum(nums) + target) // 2
    # 转化成: 从数组中选几个数作为正数，使得它们的和为x
    # 状态: dp[i][j]表示可选元素为nums[:i]，目标和为j
    # 状态转移: 如果选择当前元素, dp[i][j] += dp[i-1][j-nums[i]]，
    #         如果不选择当前元素, dp[i][j] = dp[i-1][j]
    # 初始化: dp[0] = 1, 边界: 目标值不能比总和还大，目标值不能是奇数
    if target > sum(nums): return 0
    if (sum(nums) + target) % 2: return 0
    x = (sum(nums) + target) // 2
    dp = [1] + [0] * x
    for num in nums:
        # 一定要注意范围从x开始
        for j in range(x, -1, -1):
            dp[j] += dp[j - num]
    return dp[-1]

### 动态规划Part4: 完全背包问题

In [ ]:
# 最少的硬币数目 (可以凑成目标值的最少硬币数，硬币数量无限，不能凑成返回-1)
def coinChange(coins, amount):
    # 状态: dp[i][j]表示coins[:i]组成目标值j的最少硬币个数
    # 状态转移: 如果组合中最后一个硬币选择当前硬币 dp[i][j] = dp[i][j-nums[i]] + 1
    #         如果不选择当前硬币 dp[i][j] = dp[i-1][j]
    # 初始化: dp[0][0] = 1
    # 优化: 只与当前行左边的值，或者前一行对应位置的值有关。顺序更新(因为转移时左边值是更新值)
    dp = [0] + [float(inf)] * amount
    for coin in coins:
    # 一定要注意范围要到 amount+1
        for j in range(coin, amount + 1):
            dp[j] = min(dp[j], dp[j-coin]+1)
    return dp[-1] if dp[-1] != float(inf) else -1
print("coinChange: ", coinChange([1, 2, 5], 11))

# 排列的数目 (从包含不同正整数的nums中找出总和为target的元素组合个数，顺序不同看做不同组合)
def combinationSum(nums, target):
    # 排列问题要考虑顺序，先循环背包，再循环物品
    # 状态: dp[j]表示目标和为j的排列数目。
    # 状态转移: 当前是排列的最后一个位置, 可以任选一个数字，并求和dp[j-num]
    # 初始化: 目标和为0时有1种排列
    dp = [1] + [0] * target
    for i in range(1, target + 1):
        for num in nums:
            # 保证不会越界
            if i >= num:
                dp[i] += dp[i-num]
    return dp[-1]
    